In [2]:
import textract
import pandas as pd
import numpy as np
import re

In [3]:
import tabula

In [4]:
#extraigo texto
archivo= 'tresalia\Edo Cta\JPM_PDF_por_contratar BE.pdf'

In [5]:
text = textract.process(archivo)

In [6]:
#cambio formato
encoding = 'utf-8'
text_str=text.decode(encoding)

In [7]:
#separo
pdf=re.split(r"[~\r\n]+", text_str)

In [8]:
information_pdf=pd.DataFrame(pdf).rename(columns={0:"informacion"})
information_pdf.head()

,informacion
0,NOMBRE APELLIDO P APELLIDO M DIRECCIÓN COL
1,"MEXICO, DF 00000 MEXICO"
2,For the Period 8/1/20 to 8/31/20
3,Account Summary
4,Account Number


In [9]:
#periodo
periodo_lista= []
for i in pdf:
    if re.search('Period',i):
        periodo_lista.append(i)
periodo=periodo_lista[0]

In [10]:
periodo

'\x0c\x0cFor the Period 8/1/20 to 8/31/20'

In [11]:
datos_lista=[]
for i in pdf:
    if re.search('COL',i):
        datos_lista.append(i)
        
datos_personales=datos_lista[0]

In [12]:
datos_personales

'NOMBRE APELLIDO P APELLIDO M DIRECCIÓN COL'

In [13]:
#cuenta
cuenta_lista=[]
for i in range(len(pdf)):
    if re.search('Account Number',pdf[i]):
        cuenta_lista.append(pdf[i])
        cuenta_lista.append(pdf[i+1])

In [14]:
cuenta_lista_2=[]
for i in range(len(cuenta_lista)):
    if cuenta_lista[i].startswith('Account'):
        cuenta_lista_2.append(cuenta_lista[i+1])
        

In [15]:
cuenta=cuenta_lista_2[1]
cuenta

'X99999999'

In [51]:
saldo_lista=[]
for i in range(len(pdf)):
    if re.search('beginning cash',pdf[i].lower()):
        saldo_lista.append(pdf[i+1])
        saldo_lista.append(pdf[i+2])
        saldo_lista.append(pdf[i+3])
        saldo_lista.append(pdf[i+4])
        saldo_lista.append(pdf[i+5])

In [55]:
for i in saldo_lista:
    if re.search(r'\d+',i):
        saldo_inicial=i

In [56]:
saldo_inicial

'6,146.33'

In [19]:
indice_actividad=[]
for i in range(len(pdf)):
    if re.search('Portfolio Activity Detail - U S Dollar',pdf[i]):
        indice_actividad.append(i)
    

In [20]:
tabla=pdf[indice_actividad[0]:-1]
tabla

['\x0cPortfolio Activity Detail - U S Dollar',
 'INFLOWS & OUTFLOWS',
 'Type Settle Date Selection Method 8/3 Interest Income 8/11 ACH Payment',
 '8/13 Misc Credit',
 'Total Inflows & Outflows',
 'Description',
 'DEPOSIT SWEEP INTEREST FOR 07/01/20 - 07/31/20 @ .05% RATE ON AVG COLLECTED BALANCE OF $264,551.41 AS OF 08/01/20',
 'ACH DEBIT ORIG CO NAME:CHASE CREDIT CRD ORIG ID:4760039224 DESC DATE:200810 CO ENTRY DESCR:AUTOPAY SEC:PPD TRACE#:021000029480167 EED:200810 IND ID:0000XXXXX022922 IND NAME:XXXXXXXX XXXXXXX A TRN: 2239480167TC AS OF 08/10/20',
 'FEDWIRE CREDIT VIA: THE BANK OF NEW YORK MELLON/021000018 B/O: XXXXXX XXXXXX XXXXXX XXXXX XXXXXXXX: CHASE NYC/CTR/BNF=XXXX XXXXXXX X-II MEXICO, DF 11000 MEXICO MX/ AC-000000051115 RFB=O/B BK OF NY C OBI=TRASPASO CUENTA PROPIA BBI=/C HGS/USD0,00/OCMT/USD20656,21/ TRN: 5608509226FF',
 'ACCT. X99999999 For the Period 8/1/20 to 8/31/20',
 'Quantity Cost',
 'Per Unit Amount',
 'Amount 11.22',
 '(5,833.99)',
 '2,656.21',
 '$4,833.44',
 'Accou

In [21]:
fecha_lista=[]
for i in range(len(tabla)):
    if re.search('Settle Date',tabla[i]):
        fecha_lista.append(tabla[i])
        fecha_lista.append(tabla[i+1])
        fecha_lista.append(tabla[i+2])

In [22]:
fecha_lista_2=[]
for i in fecha_lista:
    j=re.findall(r'\d+/\d+',i)
    fecha_lista_2.append(j)

In [23]:
fecha_lista_2

[['8/3', '8/11'], ['8/13'], []]

In [24]:
fechas = []
for sublist in fecha_lista_2:
    for item in sublist:
        fechas.append(item)

In [25]:
fechas

['8/3', '8/11', '8/13']

In [26]:
#tipos
tipo_lista=[]
for i in range(len(tabla)):
    if re.search('Type',tabla[i]):
        tipo_lista.append(tabla[i])
        tipo_lista.append(tabla[i+1])
        tipo_lista.append(tabla[i+2])

In [27]:
tipo_lista

['Type Settle Date Selection Method 8/3 Interest Income 8/11 ACH Payment',
 '8/13 Misc Credit',
 'Total Inflows & Outflows']

In [28]:
tipo_lista_2=[]
for i in tipo_lista:
    j=re.split(r'\d+/\d+',i)
    tipo_lista_2.append(j)

In [29]:
tipo_lista_2

[['Type Settle Date Selection Method ', ' Interest Income ', ' ACH Payment'],
 ['', ' Misc Credit'],
 ['Total Inflows & Outflows']]

In [30]:
tipos = []
for sublist in tipo_lista_2:
    for item in sublist:
        if item.startswith('Type')==False and item.startswith('Total')==False and item!='':
            print(item)
            tipos.append(item)

 Interest Income 
 ACH Payment
 Misc Credit


In [31]:
tipos

[' Interest Income ', ' ACH Payment', ' Misc Credit']

In [32]:
#descripcion
descripcion_lista=[]
for i in range(len(tabla)):
    if re.search('Description',tabla[i]):
        descripcion_lista.append(tabla[i])
        descripcion_lista.append(tabla[i+1])
        descripcion_lista.append(tabla[i+2])
        descripcion_lista.append(tabla[i+3])

In [33]:
descripciones=descripcion_lista[1:len(descripcion_lista)]

In [34]:
descripciones

['DEPOSIT SWEEP INTEREST FOR 07/01/20 - 07/31/20 @ .05% RATE ON AVG COLLECTED BALANCE OF $264,551.41 AS OF 08/01/20',
 'ACH DEBIT ORIG CO NAME:CHASE CREDIT CRD ORIG ID:4760039224 DESC DATE:200810 CO ENTRY DESCR:AUTOPAY SEC:PPD TRACE#:021000029480167 EED:200810 IND ID:0000XXXXX022922 IND NAME:XXXXXXXX XXXXXXX A TRN: 2239480167TC AS OF 08/10/20',
 'FEDWIRE CREDIT VIA: THE BANK OF NEW YORK MELLON/021000018 B/O: XXXXXX XXXXXX XXXXXX XXXXX XXXXXXXX: CHASE NYC/CTR/BNF=XXXX XXXXXXX X-II MEXICO, DF 11000 MEXICO MX/ AC-000000051115 RFB=O/B BK OF NY C OBI=TRASPASO CUENTA PROPIA BBI=/C HGS/USD0,00/OCMT/USD20656,21/ TRN: 5608509226FF']

In [35]:
#cantidad
#descripcion
cantidad_lista=[]
for i in range(len(tabla)):
    if re.search('Amount ',tabla[i]):
        cantidad_lista.append(tabla[i])
        cantidad_lista.append(tabla[i+1])
        cantidad_lista.append(tabla[i+2])

In [36]:
cantidad_lista

['Amount 11.22', '(5,833.99)', '2,656.21']

In [62]:
cantidad_lista_2=[]
for i in cantidad_lista:
    j=i.split(' ')
    cantidad_lista_2.append(j)

In [63]:
cantidad_lista_3=[]
for sublist in cantidad_lista_2:
    for item in sublist:
        cantidad_lista_3.append(item)

In [64]:
cantidad_lista_3

['Amount', '11.22', '(5,833.99)', '2,656.21']

In [70]:
movimientos=cantidad_lista_3[1:len(cantidad_lista_3)]

In [81]:
retiros=[]
depositos=[]

for i in movimientos:
    if i.startswith('('):
        retiros.append((i.replace('(','').replace(')','')))
        depositos.append(0)
    else:
        retiros.append(0)
        depositos.append(i)

In [82]:
depositos

['11.22', 0, '2,656.21']

In [83]:
retiros

[0, '5,833.99', 0]

In [86]:
from re import sub
from decimal import Decimal

In [88]:
Decimal(sub(r'[^\d.]', '','5,833.99' ))

Decimal('5833.99')

In [100]:
#cambio formato
depositos_2=[]
for i in depositos:
    value = Decimal(sub(r'[^\d.]', '', str(i)))
    depositos_2.append(int(value))
    

In [101]:
retiros_2=[]
for i in retiros:
    value = Decimal(sub(r'[^\d.]', '', str(i)))
    retiros_2.append(int(value))

In [107]:
saldo_inicial_num= int(Decimal(sub(r'[^\d.]', '', saldo_inicial)))

In [108]:
saldos=[]
saldo=saldo_inicial_num
for i,j in zip(depositos_2,retiros_2):
    saldo=int(saldo)+i-j
    saldos.append(saldo)

In [110]:
estado=pd.DataFrame(list(zip(fechas, descripciones, retiros, depositos, saldos)),
              columns=['fecha','concepto','retiro', 'deposito','saldo'])

In [112]:
estado['banco']= 'JPMorgan'

In [113]:
estado['nacionalidad']='extranjero'

In [114]:
estado['fecha de corte']=periodo

In [115]:
estado['titular']=datos_personales

In [116]:
estado['cuenta']=cuenta

In [118]:
if re.search('usd',text_str.lower()) or re.search('dolar', text_str.lower()) or re.search('dollar', text_str.lower()):
    moneda='usd'
else:
    moneda='mxn'

In [120]:
estado['divisa']=moneda

In [121]:
estado['saldo inicial']=saldo_inicial_num

In [122]:
edo_cuenta= estado[['banco','nacionalidad','fecha de corte', 'titular','cuenta','fecha','concepto','saldo inicial',
                    'retiro','deposito','saldo','divisa']]

In [124]:
edo_cuenta

,banco,nacionalidad,fecha de corte,titular,cuenta,fecha,concepto,saldo inicial,retiro,deposito,saldo,divisa
0,JPMorgan,extranjero,For the Period 8/1/20 to 8/31/20,NOMBRE APELLIDO P APELLIDO M DIRECCIÓN COL,X99999999,8/3,DEPOSIT SWEEP INTEREST FOR 07/01/20 - 07/31/20...,6146,0,11.22,6157,usd
1,JPMorgan,extranjero,For the Period 8/1/20 to 8/31/20,NOMBRE APELLIDO P APELLIDO M DIRECCIÓN COL,X99999999,8/11,ACH DEBIT ORIG CO NAME:CHASE CREDIT CRD ORIG I...,6146,"5,833.99",0,324,usd
2,JPMorgan,extranjero,For the Period 8/1/20 to 8/31/20,NOMBRE APELLIDO P APELLIDO M DIRECCIÓN COL,X99999999,8/13,FEDWIRE CREDIT VIA: THE BANK OF NEW YORK MELLO...,6146,0,"2,656.21",2980,usd


In [164]:
nombre_archivo=re.findall(r'\d+/\d+/\d+',periodo)[0]+'-jpmorgan'
nombre_archivo=nombre_archivo.replace('/','-')
nombre_archivo

'8-1-20-jpmorgan'

In [165]:
#exporto
#edo_cuenta.to_csv('tresalia\Bancos\jpmorgan\salida\\'+nombre_archivo+'.txt' , sep='\t', index=False)